In [187]:
import panel as pn
import panel.widgets as pnw
import pandas as pd
import numpy as np
from panel.interact import interact
from IPython.display import Image
from IPython.core.display import HTML
import param
from pathlib import Path 
import hvplot.pandas
from holoviews import opts
from bokeh.models import HoverTool
pn.extension()

### Dataframes

In [2]:
# Set file path
mortality_file_path = Path('Resources/Mortality.csv', header = 0)

In [3]:
# Read CSV to DataFrame
mortality_data_df = pd.read_csv(mortality_file_path)
#mortality_data_df.head()

In [4]:
# Remove Uwanted Columns from DataFrame 
mortality_data_df = mortality_data_df.drop(['Split', 'SplitSex', 'Forecast'], axis = 1)
#mortality_data_df.head()

In [5]:
# List Unique Country Codes
country_codes = mortality_data_df['CountryCode'].unique()
#print(country_codes)

In [6]:
# Create Country Dictionary
country_dict = dict(zip(country_codes, ['Australia', 'Austria', 'Belgium', 'Bulgaria', 'Canada', 'Switzerland', 'Chile', 'Czech Republic', 'Germany', 'Denmark',
                                        'Spain', 'Estonia', 'Finland', 'France', 'England and Wales', 'Northern Ireland', 'Scotland', 'Greece', 
                                        'Croatia', 'Hungary', 'Iceland', 'Israel', 'Italy', 'South Korea', 'Lithuania', 'Luxembourg', 'Latvia', 
                                        'Netherlands', 'Norway', 'New Zealand', 'Poland', 'Portugal', 'Russia', 'Slovakia', 'Slovenia', 'Sweden', 'Taiwan', 'USA']))

#country_dict

In [7]:
# Use Dictionary to Create a Country Name Column
mortality_data_df['Country']= mortality_data_df['CountryCode'].map(country_dict)

# mortality_data_df

In [8]:
# Display full name for sex
sexes = mortality_data_df['Sex'].unique()
sex_dict = dict(zip(sexes, ['Male', 'Female', 'Both']))
mortality_data_df['Sex']= mortality_data_df['Sex'].map(sex_dict)

#mortality_data_df

In [9]:
# Create a Datetime Dataframe
mortality_data_dt_df = mortality_data_df
mortality_data_dt_df['Date'] = pd.to_datetime(mortality_data_df.Year.astype(str), format='%Y') + \
             pd.to_timedelta(mortality_data_df.Week.mul(7).astype(str) + ' days')
# Remove the Year and Week Columns
mortality_data_dt_df = mortality_data_dt_df[['CountryCode', 'Country', 'Date', 'Sex', 'D0_14', 'D15_64', 'D65_74','D75_84', 'D85p',
                                          'DTotal', 'R0_14', 'R15_64', 'R65_74', 'R75_84','R85p', 'RTotal']]

mortality_data_dt_df.columns = ['CountryCode','Country', 'Date', 'Sex',
                                          'Deaths: 0-14 yrs',
                                          'Deaths: 15-64 yrs',
                                          'Deaths: 65-74 yrs',
                                          'Deaths: 75-84 yrs',
                                          'Deaths: 85+ yrs',
                                          'Deaths: Total',
                                          'Death Rate: 0-14 yrs',
                                          'Death Rate: 15-64 yrs',
                                          'Death Rate: 65-74 yrs',
                                          'Death Rate: 75-84 yrs',
                                          'Death Rate: 85+ yrs',
                                          'Death Rate: Total'
                                          ]
mortality_data_dt_df

,CountryCode,Country,Date,Sex,Deaths: 0-14 yrs,Deaths: 15-64 yrs,Deaths: 65-74 yrs,Deaths: 75-84 yrs,Deaths: 85+ yrs,Deaths: Total,Death Rate: 0-14 yrs,Death Rate: 15-64 yrs,Death Rate: 65-74 yrs,Death Rate: 75-84 yrs,Death Rate: 85+ yrs,Death Rate: Total
0,AUS2,Australia,2015-01-08,Male,5.037600,210.962400,204.0,398.0,394.0,1212.0,0.000113,0.001395,0.010716,0.041683,0.119154,0.005326
1,AUS2,Australia,2015-01-08,Female,6.758007,141.241993,154.0,323.0,676.0,1301.0,0.000160,0.000929,0.007869,0.028785,0.118644,0.005641
2,AUS2,Australia,2015-01-08,Both,11.795607,352.204393,358.0,721.0,1070.0,2513.0,0.000136,0.001161,0.009273,0.034714,0.118831,0.005484
3,AUS2,Australia,2015-01-15,Male,5.648218,166.351782,216.0,343.0,399.0,1130.0,0.000127,0.001100,0.011347,0.035923,0.120666,0.004966
4,AUS2,Australia,2015-01-15,Female,6.983274,149.016726,147.0,290.0,646.0,1239.0,0.000166,0.000980,0.007511,0.025844,0.113378,0.005372
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103159,USA,USA,2021-05-14,Female,226.000000,5334.000000,4860.0,6333.0,9075.0,25828.0,0.000403,0.002610,0.014131,0.034308,0.108929,0.008031
103160,USA,USA,2021-05-14,Both,515.000000,14814.000000,11325.0,13071.0,14911.0,54636.0,0.000449,0.003626,0.017570,0.039663,0.113996,0.008622
103161,USA,USA,2021-05-21,Male,271.000000,9088.000000,6445.0,6657.0,5776.0,28237.0,0.000463,0.004451,0.021436,0.045924,0.121621,0.009048
103162,USA,USA,2021-05-21,Female,195.000000,5401.000000,4918.0,6309.0,9121.0,25944.0,0.000348,0.002642,0.014300,0.034178,0.109482,0.008067


In [93]:
# Set the Country Columns as the index
mortality_data_country_indexed = mortality_data_dt_df.set_index(mortality_data_dt_df['Country'])
mortality_data_country_indexed.drop(columns=['Country'], inplace=True)
mortality_data_country_indexed.columns = ['CountryCode', 'Date', 'Sex',
                                          'Deaths: 0-14 yrs',
                                          'Deaths: 15-64 yrs',
                                          'Deaths: 65-74 yrs',
                                          'Deaths: 75-84 yrs',
                                          'Deaths: 85+ yrs',
                                          'Deaths: Total',
                                          'Death Rate: 0-14 yrs',
                                          'Death Rate: 15-64 yrs',
                                          'Death Rate: 65-74 yrs',
                                          'Death Rate: 75-84 yrs',
                                          'Death Rate: 85+ yrs',
                                          'Death Rate: Total'
                                          ]
mortality_data_country_indexed

,CountryCode,Date,Sex,Deaths: 0-14 yrs,Deaths: 15-64 yrs,Deaths: 65-74 yrs,Deaths: 75-84 yrs,Deaths: 85+ yrs,Deaths: Total,Death Rate: 0-14 yrs,Death Rate: 15-64 yrs,Death Rate: 65-74 yrs,Death Rate: 75-84 yrs,Death Rate: 85+ yrs,Death Rate: Total
Country,,,,,,,,,,,,,,,
Australia,AUS2,2015-01-08,Male,5.037600,210.962400,204.0,398.0,394.0,1212.0,0.000113,0.001395,0.010716,0.041683,0.119154,0.005326
Australia,AUS2,2015-01-08,Female,6.758007,141.241993,154.0,323.0,676.0,1301.0,0.000160,0.000929,0.007869,0.028785,0.118644,0.005641
Australia,AUS2,2015-01-08,Both,11.795607,352.204393,358.0,721.0,1070.0,2513.0,0.000136,0.001161,0.009273,0.034714,0.118831,0.005484
Australia,AUS2,2015-01-15,Male,5.648218,166.351782,216.0,343.0,399.0,1130.0,0.000127,0.001100,0.011347,0.035923,0.120666,0.004966
Australia,AUS2,2015-01-15,Female,6.983274,149.016726,147.0,290.0,646.0,1239.0,0.000166,0.000980,0.007511,0.025844,0.113378,0.005372
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
USA,USA,2021-05-14,Female,226.000000,5334.000000,4860.0,6333.0,9075.0,25828.0,0.000403,0.002610,0.014131,0.034308,0.108929,0.008031
USA,USA,2021-05-14,Both,515.000000,14814.000000,11325.0,13071.0,14911.0,54636.0,0.000449,0.003626,0.017570,0.039663,0.113996,0.008622
USA,USA,2021-05-21,Male,271.000000,9088.000000,6445.0,6657.0,5776.0,28237.0,0.000463,0.004451,0.021436,0.045924,0.121621,0.009048


In [91]:
# Set the Date Column as the index
mortality_data_dt_indexed = mortality_data_dt_df.set_index(mortality_data_dt_df['Date'])
mortality_data_dt_indexed.drop(columns=['Date'], inplace=True)
mortality_data_dt_indexed.columns = ['Country', 'CountryCode', 'Sex',
                                          'Deaths: 0-14 yrs',
                                          'Deaths: 15-64 yrs',
                                          'Deaths: 65-74 yrs',
                                          'Deaths: 75-84 yrs',
                                          'Deaths: 85+ yrs',
                                          'Deaths: Total',
                                          'Death Rate: 0-14 yrs',
                                          'Death Rate: 15-64 yrs',
                                          'Death Rate: 65-74 yrs',
                                          'Death Rate: 75-84 yrs',
                                          'Death Rate: 85+ yrs',
                                          'Death Rate: Total'
                                          ]

#mortality_data_dt_indexed

In [167]:
# Transpose the Dataframe
mortality_data_country_indexed_transposed = mortality_data_country_indexed.T
#mortality_data_country_indexed_transposed

In [159]:
date = pd.DataFrame(mortality_data_country_indexed_transposed.loc["Date"])
date['Sex'] = mortality_data_country_indexed_transposed.loc["Sex"]
#date

In [171]:
country_selector = pn.widgets.Select(name='Country Selector', value='USA',
    options=list(mortality_data_dt_df.Country.unique()))
country_selector

Select(name='Country Selector', options=['Australia', ...], value='USA')

### Widgets

In [192]:
@pn.depends(country_selector)
def country_loc(country_selector):
    country = mortality_data_country_indexed.loc[country_selector]
    return country

#pn.Column(pn.WidgetBox(country_selector), country_loc, width = 1000)

In [193]:
year_slider = pn.widgets.IntRangeSlider(name='Years Slider', width=300, start=2010, end=2022, value=(2015, 2022), value_throttled=(2015, 2022))
#year_slider

In [194]:
# Dynamic markup title for slider
@pn.depends(year_slider)
def year_range(year_slider):
    return '### Yearly Data Between {start} —  {end}'.format(start=year_slider[0], end=year_slider[1])

In [195]:
#pn.Row(year_range).show()

In [208]:

sex_selector = pn.widgets.Select(name='Sex Selector', value='Both',
    options=list(mortality_data_dt_df.Sex.unique()))
@pn.depends(sex_selector)
def sex_filter(sex_selector):
    sex = mortality_data_country_indexed[mortality_data_country_indexed['Sex'] == sex_selector]
    return sex

In [209]:
pn.Row(sex_filter).show()

Launching server at http://localhost:64534


In [220]:
stats_selector = pn.widgets.Select(name='Statistic', value = 'Death Rate: Total', options=['Deaths: 0-14 yrs',
                                          'Deaths: 15-64 yrs',
                                          'Deaths: 65-74 yrs',
                                          'Deaths: 75-84 yrs',
                                          'Deaths: 85+ yrs',
                                          'Deaths: Total',
                                          'Death Rate: 0-14 yrs',
                                          'Death Rate: 15-64 yrs',
                                          'Death Rate: 65-74 yrs',
                                          'Death Rate: 75-84 yrs',
                                          'Death Rate: 85+ yrs',
                                          'Death Rate: Total'])
@pn.depends(stats_selector)
def stats_filter(stats_selector):
    stat = mortality_data_country_indexed[stats_selector]
    return stat


In [222]:
pn.Row(stats_selector, stats_filter).show()

Launching server at http://localhost:60725


## Dynamic Plots

In [225]:
# Dynamic plot with multiple x values

# Y value multiselectors
stats_multiselector = pn.widgets.MultiSelect(name='Statistic', value = ['Death Rate: Total'], options=['Deaths: 0-14 yrs',
                                          'Deaths: 15-64 yrs',
                                          'Deaths: 65-74 yrs',
                                          'Deaths: 75-84 yrs',
                                          'Deaths: 85+ yrs',
                                          'Deaths: Total',
                                          'Death Rate: 0-14 yrs',
                                          'Death Rate: 15-64 yrs',
                                          'Death Rate: 65-74 yrs',
                                          'Death Rate: 75-84 yrs',
                                          'Death Rate: 85+ yrs',
                                          'Death Rate: Total'])
country_multiselector = pn.widgets.MultiSelect(name='Country Selector', value=['USA'],
    options=list(mortality_data_dt_df.Country.unique()))

# X value selectors
stats_selector = pn.widgets.Select(name='Statistic', value = 'Death Rate: Total', options=['Deaths: 0-14 yrs',
                                          'Deaths: 15-64 yrs',
                                          'Deaths: 65-74 yrs',
                                          'Deaths: 75-84 yrs',
                                          'Deaths: 85+ yrs',
                                          'Deaths: Total',
                                          'Death Rate: 0-14 yrs',
                                          'Death Rate: 15-64 yrs',
                                          'Death Rate: 65-74 yrs',
                                          'Death Rate: 75-84 yrs',
                                          'Death Rate: 85+ yrs',
                                          'Death Rate: Total'])

country_selector = pn.widgets.Select(name='Country Selector', value='USA',
    options=list(mortality_data_dt_df.Country.unique()))

sex_selector = pn.widgets.Select(name='Sex Selector', value='Both',
    options=list(mortality_data_dt_df.Sex.unique()))

sex_selector_2 = pn.widgets.Select(name='Sex Selector', value='Both',
    options=list(mortality_data_dt_df.Sex.unique()))

# Year Range Sliders
year_slider = pn.widgets.IntRangeSlider(name='Years Slider', width=300, start=2010, end=2022, value=(2015, 2022), value_throttled=(2015, 2022))

year_slider_2 = pn.widgets.IntRangeSlider(name='Years Slider', width=300, start=2010, end=2022, value=(2015, 2022), value_throttled=(2015, 2022))

@pn.depends(year_slider, country_selector, sex_selector)
def plot_multi_stats(year_slider, country_selector, sex_selector):
    sex = mortality_data_country_indexed[mortality_data_country_indexed['Sex'] == sex_selector]
    country = sex.loc[country_selector]
    years_df = country[country.Date.dt.year.between(year_slider[0], year_slider[1])]
    plot = years_df.hvplot(title='Deaths by Country and Sex', 
                           x = "Date", 
                           y = stats_multiselector,
                           value_label = "Deaths and/or Death Rate", 
                           invert = False, 
                           height = 400, 
                           #groupby = ['Sex'],
                           widget_location = 'bottom_left')
    return plot

@pn.depends(year_slider_2, stats_selector, sex_selector_2)
def plot_multi_country(year_slider, stats_selector, sex_selector_2):
    sex = mortality_data_country_indexed[mortality_data_country_indexed['Sex'] == sex_selector_2]
    stat = sex[stats_selector]
    years_df = stat[stat.Date.dt.year.between(year_slider[0], year_slider[1])]
    plot = years_df.hvplot(title='Deaths by Country and Sex', 
                           x = "Date", 
                           y = country_multiselector,
                           value_label = "Deaths and/or Death Rate", 
                           invert = False, 
                           height = 400, 
                           #groupby = ['Sex'],
                           widget_location = 'bottom_left')
    return plot

custom = pn.Column(pn.Row(pn.WidgetBox(stats_multiselector, year_slider, country_selector, sex_selector), plot_multi_stats), 
                  pn.Row(pn.WidgetBox(country_multiselector, year_slider_2, stats_selector, sex_selector_2), plot_multi_country))
custom.show()

AttributeError: 'Series' object has no attribute 'Date'

In [137]:
help(hvplot.plotting.core)

Help on module hvplot.plotting.core in hvplot.plotting:

NAME
    hvplot.plotting.core

CLASSES
    builtins.object
        hvPlotBase
            hvPlotTabular
                hvPlot
    
    class hvPlot(hvPlotTabular)
     |  hvPlot(data, custom_plots={}, **metadata)
     |  
     |  Method resolution order:
     |      hvPlot
     |      hvPlotTabular
     |      hvPlotBase
     |      builtins.object
     |  
     |  Methods defined here:
     |  
     |  contour(self, x=None, y=None, z=None, colorbar=True, *, alpha, cmap, color, hover_alpha, hover_color, hover_line_alpha, hover_line_color, line_alpha, line_cap, line_color, line_dash, line_join, line_width, muted, muted_alpha, muted_color, muted_line_alpha, muted_line_color, nonselection_alpha, nonselection_color, nonselection_line_alpha, nonselection_line_color, selection_alpha, selection_color, selection_line_alpha, selection_line_color, visible, levels, logz, width, height, shared_axes, grid, legend, rot, xlim, ylim, xticks, yt

In [ ]:
# mplot = pn.widgets.Select(name='# of Countries', options=['One Country', 'Two Countries', 'Three Countries', 'Four Countries'])

# # Multiple y values for different plots
# y2 = pn.widgets.MultiSelect(name='Statistic', options=['Deaths: 0-14 yrs',
#                                           'Deaths: 15-64 yrs',
#                                           'Deaths: 65-74 yrs',
#                                           'Deaths: 75-84 yrs',
#                                           'Deaths: 85+ yrs',
#                                           'Deaths: Total',
#                                           'Death Rate: 0-14 yrs',
#                                           'Death Rate: 15-64 yrs',
#                                           'Death Rate: 65-74 yrs',
#                                           'Death Rate: 75-84 yrs',
#                                           'Death Rate: 85+ yrs',
#                                           'Death Rate: Total'])

# y3 = pn.widgets.MultiSelect(name='Statistic', options=['Deaths: 0-14 yrs',
#                                           'Deaths: 15-64 yrs',
#                                           'Deaths: 65-74 yrs',
#                                           'Deaths: 75-84 yrs',
#                                           'Deaths: 85+ yrs',
#                                           'Deaths: Total',
#                                           'Death Rate: 0-14 yrs',
#                                           'Death Rate: 15-64 yrs',
#                                           'Death Rate: 65-74 yrs',
#                                           'Death Rate: 75-84 yrs',
#                                           'Death Rate: 85+ yrs',
#                                           'Death Rate: Total'])

# y4 = pn.widgets.MultiSelect(name='Statistic', options=['Deaths: 0-14 yrs',
#                                           'Deaths: 15-64 yrs',
#                                           'Deaths: 65-74 yrs',
#                                           'Deaths: 75-84 yrs',
#                                           'Deaths: 85+ yrs',
#                                           'Deaths: Total',
#                                           'Death Rate: 0-14 yrs',
#                                           'Death Rate: 15-64 yrs',
#                                           'Death Rate: 65-74 yrs',
#                                           'Death Rate: 75-84 yrs',
#                                           'Death Rate: 85+ yrs',
#                                           'Death Rate: Total'])

# # Plot functions for multiple plots
# @pn.depends(year_slider.param.value_throttled)
# def plot_bar_2(year_slider):
#     years_df_2 = mortality_data_country_indexed[mortality_data_country_indexed.Date.dt.year.between(year_slider[0], year_slider[1])]
#     return years_df_2.hvplot(title='Deaths by Country and Sex', 
#                            x = "Date", 
#                            y = y2, 
#                            value_label = "Deaths and/or Death Rate", 
#                            invert = False, 
#                            height = 400, 
#                            groupby = ["CountryCode", "Sex"], 
#                            widget_location = 'left_top')

# @pn.depends(year_slider.param.value_throttled)
# def plot_bar_3(year_slider):
#     years_df_3 = mortality_data_country_indexed[mortality_data_country_indexed.Date.dt.year.between(year_slider[0], year_slider[1])]
#     return years_df_3.hvplot(title='Deaths by Country and Sex', 
#                            x = "Date", 
#                            y = y3, 
#                            value_label = "Deaths and/or Death Rate", 
#                            invert = False, 
#                            height = 400, 
#                            groupby = ["CountryCode", "Sex"], 
#                            widget_location = 'left_top')

# @pn.depends(year_slider.param.value_throttled)
# def plot_bar_4(year_slider):
#     years_df_4 = mortality_data_country_indexed[mortality_data_country_indexed.Date.dt.year.between(year_slider[0], year_slider[1])]
#     return years_df_4.hvplot(title='Deaths by Country and Sex', 
#                            x = "Date", 
#                            y = y4, 
#                            value_label = "Deaths and/or Death Rate", 
#                            invert = False, 
#                            height = 400, 
#                            groupby = ["CountryCode", "Sex"], 
#                            widget_location = 'left_top',)


In [ ]:
# # Placing all plots in their own widget
# plot_widget_1 = pn.WidgetBox(y, plot_bar)
# plot_widget_2 = pn.WidgetBox(y2, plot_bar_2)
# plot_widget_3 = pn.WidgetBox(y3, plot_bar_3)
# plot_widget_4 = pn.WidgetBox(y4, plot_bar_4)

# new_custom = pn.Column(mplot, year_slider, plot_widget_1, plot_widget_2, plot_widget_3, plot_widget_4)

In [ ]:
# Event update for enabling multiple plots
# def update(event):
#     if mplot.value == 'One Country':
#         plot_widget_1.disabled = False
#         plot_widget_2.disabled = True
#         plot_widget_3.disabled = True
#         plot_widget_4.disabled = True
#     if mplot.value == 'Two Countries':
#         plot_widget_1.disabled = False
#         plot_widget_2.disabled = False
#         plot_widget_3.disabled = True
#         plot_widget_4.disabled = True
#     if mplot.value == 'Three Countries':
#         plot_widget_1.disabled = False
#         plot_widget_2.disabled = False
#         plot_widget_3.disabled = False
#         plot_widget_4.disabled = True
#     else:
#         plot_widget_1.disabled = False
#         plot_widget_2.disabled = False
#         plot_widget_3.disabled = False
#         plot_widget_4.disabled = False
        
# mplot.param.watch(update, 'value');

In [ ]:
# # Dashboard elements
# dash_title = "# Country Mortality Analysis"

# # Dashboard description
# dash_desc = "Our project aims to visualize different metrics of COVID-19, specifically focusing on excess death"
# text = "Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum."

# header_box = pn.WidgetBox(dash_title, 
#                           dash_desc, 
#                           text)

# main_box = pn.WidgetBox(year_range, new_custom)

In [ ]:
dashboard = pn.Row(header_box, main_box, sizing_mode="stretch_width")

In [ ]:
dashboard